# <font color = red>1. 실험 내용 전체 요약</font>
* mushroom data에 대해 4가지 모델(OneR, Logistic Regression, MLP, Random Forest)을 적용하였습니다.
* parameters 조정 결과 Logistic Regression에서 penalty='none'으로 조정하는 것이 거의 무조건 성능 향상을 보였습니다.
* MLP와 Random Forest는 default 모델에서 1.0의 성능이었습니다. parameters 조정을 시도해 봤으나, 성공적이지 않았습니다.
* feature selection은 성능 하락의 원인이 되었습니다.

# <font color = red>2. 데이터에 대한 설명 
* url: https://drive.google.com/uc?export=download&id=1u4PEyVWUPODNbCSMCVHDot_8sCy4V8Sk
* 위의 url이 안 될 경우: https://archive.ics.uci.edu/ml/datasets/Mushroom
* mushroom data set을 선택했습니다. feature는 총 22개고, instance의 수는 총 8124개입니다.
* 22개의 class를 바탕으로 예측할 class는 "먹을 수 있는지"에 관한 것이며 e(edible)와 p(poisonous)로 구분됩니다.
    
    
* 데이터에 대한 보다 자세한 설명은, 순서상 "3. (2) 데이터 분포 파악 및 전처리"에 적었으니 참고 바랍니다.

In [143]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
path = "https://drive.google.com/uc?export=download&id=1u4PEyVWUPODNbCSMCVHDot_8sCy4V8Sk"
data = pd.read_csv(path)
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


# <font color = red>3. 실험 설계 방법

### <font color = green>3. (1) 선정한 모델 및 선정 이유
* 선정한 모델:
** OneR
** Logistic Regression
** MLP
** Random Forest


* 선정한 이유: 

** OneR: 당연한 말이지만, OneR이 좋은 성능을 낼 거라고 기대하고 선택한 것은 아닙니다. 그래도, OneR을 사용하면 가장 구분력 있는 feature를 찾을 수 있을 것이라고 기대했습니다. 이후 모델 선정 및 feature selection 등에 도움을 받기 위해, 데이터 특성에 대한 감을 잡기 위해 선택했습니다.


** Logistic Regression: 가장 큰 이유는 binary classification이기 때문입니다. 데이터에서 feature의 수가 다소 많다고 느껴지나, 어쨌든 class는 2개인 데이터입니다. 그렇기 때문에 가장 전형적인 binary classification 방법인 Logistic Regression을 선택했습니다.
* 물론 Logistic Regression은 linear하게 classification을 진행하기 때문에 보통 최고의 성능을 보이진 않을 것으로 예상합니다. 그러나 최소한 feature가 linear한 방법으로 충분히 잘 구분되는지는 파악할 수 있을 것입니다. 만약 linear classification으로 충분하지 않은 결가가 나온다면, 이후 좀더 복잡한 non-linear 모델을 통해 성능 향상을 노리겠습니다.

** MLP: MLP의 경우 전형적인 non-linear classfication 모델이기 때문에 선택했습니다. Logistic Regression에서 충분한 성능이 나타나지 않는다면 아마 주된 이유는 모델이 linear하기 때문일 것입니다. 그 문제를 해결하기 위해 non-linear하면서도 무난한 MLP를 골랐습니다.

** Random Forest: 수업 시간에 배운 바에 의하면, 보통 전형적인 기계학습 중에서 거의 최고의 성능을 보이는 모델 중에 하나가 Random Forest라고 들었습니다. MLP보다 월등한 성능을 기대하며 선택했습니다. 

### <font color = green>3. (2) 데이터 분포 파악 및 전처리
* 우선 describe method를 바탕으로 데이터의 특성을 최대한 파악하고자 했습니다. 우선 가장 눈에 띄는 점은 class가 binary고, freq값이 8124의 절반 근처 4208이라는 점이었습니다. class가 잘 balanced된 데이터라는 사실을 알 수 있었습니다.


* 그 다음 눈에 띄는 점은 freq값이 과도하게 높은 feature들이었습니다. gill-attacthment는 7914, gill-spacing은 6812, veil-type은 8124, veil color은 7924, ring-number는 7488의 freq를 갖고 있었습니다. class가 e: 4208, p: 3916인 데이터에 대해 위와 같이 freq가 6000이 넘어가는 feature들은 높은 확률로 irrelevant할 것이라고 추측할 수 있었습니다. 그래도 일단 임의로 제거하진 않았고, 이럴 것 같다는 경향성만 파악하고 끝냈습니다.


* 다만 종류가 1개, freq가 8124인 feature, veil-type은 무조건 확실히 필요 없는 feature일 것입니다. 구분 능력이 말 그대로 전혀 없습니다. 이 feature만큼은 제 판단만을 근거로 임의로 제거해도 된다고 생각했습니다.

In [2]:
# describe 함수를 통해 데이터의 분포를 파악했습니다.
data.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [3]:
# 필요 없는 feature인 veil-type, 그리고 na값을 제거했습니다.
data = data.drop(["veil-type"], axis=1)
data = data.dropna()

* 또한, head와 describe의 결과 모든 feature가 object type이라는 사실도 파악할 수 있습니다. 따라서 모든 feature에 label encoding또는 one hot encoding을 해서 숫자로 바꿔야 합니다.
* label encoding을 하겠습니다.

In [4]:
from sklearn.preprocessing import LabelEncoder

for i in data.columns:
    data[i] = LabelEncoder().fit_transform(data[i])
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,2,7,7,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,2,7,7,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,2,7,7,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,2,7,7,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,2,7,7,2,1,0,3,0,1


### <font color = green> 3. (3): Default 모델 훈련
* OneR 모델: OneR 모델을 생성하여 어떠한 feature가 단독으로 가장 구분력이 좋은지 파악하고자 했습니다. 우선 best feature를 선택해야 하기 때문에 각 feature의 total errors를 계산했습니다. 그리고, 해당 total errors를 argmin하는 feature를 best feature로 선정하였습니다.
* 선택된 best feature에 대해서 OneR 모델을 훈련하였고, 결과는 다음과 같습니다.

In [5]:
from collections import Counter
total_errors = []

for col in data.columns[2:]:
    error = 0
    for val in data[col].unique():
        length = len(data[data[col]==val])
        error += (length - Counter(data[data[col]==val]['class']).most_common()[0][1])
    total_errors.append(error)

In [6]:
best_feature = data.columns[np.argmin(total_errors)]
print(best_feature)

cap-color


In [7]:
oneRules = []
for val in data[best_feature].unique():
    print(f"{best_feature} : {val}","-> ",end='')
    print(Counter(data[data[best_feature]==val]['class']).most_common()[0][0])
    oneRules.append((best_feature, val, 
                    Counter(data[data[best_feature]==val]['class']).most_common()[0][0]))

cap-color : 4 -> 0
cap-color : 9 -> 1
cap-color : 8 -> 0
cap-color : 3 -> 0
cap-color : 2 -> 1
cap-color : 5 -> 1
cap-color : 0 -> 1
cap-color : 7 -> 0
cap-color : 1 -> 0
cap-color : 6 -> 0


In [8]:
oneRules

[('cap-color', 4, 0),
 ('cap-color', 9, 1),
 ('cap-color', 8, 0),
 ('cap-color', 3, 0),
 ('cap-color', 2, 1),
 ('cap-color', 5, 1),
 ('cap-color', 0, 1),
 ('cap-color', 7, 0),
 ('cap-color', 1, 0),
 ('cap-color', 6, 0)]

* Logistic Regression / MLP / Random Forest 훈련
* warning message 방지를 위해 부득이하게 Logistic Regression에서 max_iter=1000을 설정하였습니다.

In [25]:
y = data[['class']]
x = data.drop(columns = ['class'])

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
LR = LogisticRegression(max_iter = 1000)
MLP = MLPClassifier()
RF = RandomForestClassifier()

# logistic regression model 생성
mdl_LR = LR.fit(x_train, y_train.values.ravel())

# MLP 모델 생성
mdl_MLP = MLP.fit(x_train, y_train.values.ravel())

# Random Forest 모델 생성
mdl_RF = RF.fit(x_train, y_train.values.ravel())

In [26]:
# 각 모델의 점수 계산
print("Logistic Regression score: ", mdl_LR.score(x_test, y_test))
print("MLP score: ", mdl_MLP.score(x_test, y_test))
print("Random Forest score: ", mdl_RF.score(x_test, y_test))

Logistic Regression score:  0.9495488105004102
MLP score:  1.0
Random Forest score:  1.0


* 긍정적인 현상이긴 하나, 조금은 당황스럽게도 모든 default모델에서 굉장히 좋은 점수가 나왔습니다. 특히 MLP와 Random Forest의 경우에는 1.0점이라는 완벽한 점수가 나왔습니다. 그럼에도 MLP와 Random Forest에 대해 뒤에서 parameters를 조정해보긴 하겠습니다. 점수가 떨어지지 않는지만 확인해보면 되겠지요.


* 그렇다면 Logistic Regression을 중점적으로 모델의 성능을 높여보도록 하겠습니다. 또한 feature selection을 거쳐서 최고의 feature로 선택된 것에 대하여 OneR도 다시 적용해보겠습니다.


* 혹시 모르니 일단 cross validation의 score도 출력해보겠습니다.

In [29]:
from sklearn.model_selection import cross_val_score

score1 = cross_val_score(LR, x_train, y_train.values.ravel(), cv = 5)
score2 = cross_val_score(MLP, x_train, y_train.values.ravel(), cv = 5)
score3 = cross_val_score(RF, x_train, y_train.values.ravel(), cv = 5)
print("cv_LR score: ", score1)
print("cv_MLP score: ", score2)
print("cv_RF score: ", score3)

cv_LR score:  [0.93760984 0.93667546 0.96657872 0.93931398 0.94283201]
cv_MLP score:  [0.99912127 1.         1.         1.         1.        ]
cv_RF score:  [1. 1. 1. 1. 1.]


* cross validation의 score도 전반적으로 유사하게 나왔습니다. Logistic Regression만 조금 편차가 있긴 합니다. 데이터의 variation이 바뀌어도 MLP와 RF는 사실상 1.0의 score을 유지하고 있다는 사실을 알 수 있습니다.

### <font color = green> 3. (4): parameters 조정
* Logistic Regression의 성능을 높이기 위해 parameters를 조정하겠습니다. MLP와 RF에 관해서도 조정을 하되, 점수가 떨어지지 않는지만 관찰하겠습니다.


##### Logistic Regression에서 조정할 parameters:
* penalty: penalty를 주는 norm 방법 선택
* max_iter: 모델 iteration의 최대 횟수 제한


* 아래의 반복문을 통해 max_iter을 800에서 1400까지 조정해보겠고, 동시에 penalty를 'none'과 'l2'로 바꿔보겠습니다.

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

for i in range(800, 1500, 100):
    for j in ['none', 'l2']:
        LR = LogisticRegression(max_iter=i, penalty=j)
        mdl_LR = LR.fit(x_train, y_train.values.ravel())
        print("max_iter = ", str(i), "/ penalty = ", j, "\nscore: ", mdl_LR.score(x_test, y_test), "\n")


max_iter =  800 / penalty =  none 
score:  0.9692370795734209 

max_iter =  800 / penalty =  l2 
score:  0.950369155045119 

max_iter =  900 / penalty =  none 
score:  0.9692370795734209 

max_iter =  900 / penalty =  l2 
score:  0.950369155045119 

max_iter =  1000 / penalty =  none 
score:  0.9692370795734209 

max_iter =  1000 / penalty =  l2 
score:  0.950369155045119 

max_iter =  1100 / penalty =  none 
score:  0.9692370795734209 

max_iter =  1100 / penalty =  l2 
score:  0.950369155045119 

max_iter =  1200 / penalty =  none 
score:  0.9692370795734209 

max_iter =  1200 / penalty =  l2 
score:  0.950369155045119 

max_iter =  1300 / penalty =  none 
score:  0.9692370795734209 

max_iter =  1300 / penalty =  l2 
score:  0.950369155045119 

max_iter =  1400 / penalty =  none 
score:  0.9692370795734209 

max_iter =  1400 / penalty =  l2 
score:  0.950369155045119 



parameters를 여러 차례 조정해본 결과 가장 좋았던 조정은,
* penalty = none

이었습니다. max_iter을 꽤 넓은 범위인 800에서 1400까지 돌려봤으나, 성능의 차이가 "전혀" 나타나지 않았습니다. 

반면 penalty = 'none'의 경우 항상 0.969라는 성능 향상을 보장했습니다. max_iter에 상관 없이 말입니다.

위의 조정을 근거로 최적의 parameters 조정을 "max_iter=1000, penalty='none'"으로 결정하겠습니다. max_iter의 경우 1000으로 설정할 근거는 없으나 그냥 깔끔한 숫자라서 선택했습니다.

##### MLP에서 조정할 parameters:
* hidden_layer_sizes: hidden layer에 있는 neuron의 수를 결정합니다.
* activation: 어떠한 activation function을 쓸지 선택합니다. default는 relu입니다.
* learning_rate_init: learning rate의 최초 값을 설정합니다. default는 0.001입니다.

In [53]:
for i in range(100, 300, 100):
    for j in ['logistic', 'relu', 'tanh']:
        for k in np.arange(0.001, 0.01, 0.002):
            MLP = MLPClassifier(hidden_layer_sizes=i, activation=j,
                                     learning_rate_init=k)
            mdl_MLP = MLP.fit(x_train, y_train.values.ravel())
            print("hidden_layer_sizes = ", str(i), "/ activation = ", j, "/ learning_rate_init =", str(k),
                  "\nscore: ", mdl_MLP.score(x_test, y_test), "\n")


hidden_layer_sizes =  100 / activation =  logistic / learning_rate_init = 0.001 
score:  1.0 

hidden_layer_sizes =  100 / activation =  logistic / learning_rate_init = 0.003 
score:  1.0 

hidden_layer_sizes =  100 / activation =  logistic / learning_rate_init = 0.005 
score:  1.0 

hidden_layer_sizes =  100 / activation =  logistic / learning_rate_init = 0.007 
score:  1.0 

hidden_layer_sizes =  100 / activation =  logistic / learning_rate_init = 0.009000000000000001 
score:  1.0 

hidden_layer_sizes =  100 / activation =  relu / learning_rate_init = 0.001 
score:  1.0 

hidden_layer_sizes =  100 / activation =  relu / learning_rate_init = 0.003 
score:  1.0 

hidden_layer_sizes =  100 / activation =  relu / learning_rate_init = 0.005 
score:  1.0 

hidden_layer_sizes =  100 / activation =  relu / learning_rate_init = 0.007 
score:  1.0 

hidden_layer_sizes =  100 / activation =  relu / learning_rate_init = 0.009000000000000001 
score:  1.0 

hidden_layer_sizes =  100 / activation =

* 많은 조정을 해봤으나, 어떠한 조정에서도 점수가 떨어지는 경우가 없었습니다. 이 정도면 MLP는 mushroom data에 관해 최적의 모델 중 하나라고 결론 내릴 수 있다고 생각합니다.
* 또한 특별히 최적의 parameters를 찾지 못했으니, 최적의 경우를 default로 두고 넘어 가겠습니다.

##### Random Forest에서 조정할 parameters:
* n_estimators: forest에 사용되는 tree의 수입니다. 이것을 줄여보기도 하면서 점수의 하락이 발생하는지 관찰해보겠습니다.
* min_samples_split: forest 내 각 tree를 pruning하기 위해 사용하는 parameters입니다. default=2인데, 이것의 값을 늘려가며, 각 tree를 pruning했을 때의 영향이 긍정적일지, 부정적일지 판단해보겠습니다.

In [61]:
for i in range(10, 40, 10):
    for j in range(10, 40, 10):
        RF = RandomForestClassifier(n_estimators=i, min_samples_split=j)
        mdl_RF = RF.fit(x_train, y_train.values.ravel())
        print("n_estimators = ", i, "/ min_samples_split = ", j, "\nscore: ", mdl_RF.score(x_test, y_test), "\n")


n_estimators =  10 / min_samples_split =  10 
score:  1.0 

n_estimators =  10 / min_samples_split =  20 
score:  1.0 

n_estimators =  10 / min_samples_split =  30 
score:  1.0 

n_estimators =  20 / min_samples_split =  10 
score:  1.0 

n_estimators =  20 / min_samples_split =  20 
score:  1.0 

n_estimators =  20 / min_samples_split =  30 
score:  1.0 

n_estimators =  30 / min_samples_split =  10 
score:  1.0 

n_estimators =  30 / min_samples_split =  20 
score:  1.0 

n_estimators =  30 / min_samples_split =  30 
score:  1.0 



* 역시 Random Forest도 parameters에 상관 없이 항상 1.0의 점수를 보였습니다. MLP와 함께 Random Forest역시 mushroom data에 대한 최적의 모델이라고 결론 내리면 좋겠습니다.
* 또한 최적의 parameters가 딱히 없기 때문에, 최적의 경우를 그냥 default로 두고 넘어 가겠습니다.

### <font color = green> 3. (5): feature selection
feature selection의 목표는 다음과 같습니다.
* Logistic Regression의 성능을 조금이라도 더 올리기,
*  OneR에 다른 feature를 적용하여 실행해보기.

feature selection의 방법으로는 RFE를 사용하겠습니다. 우선 n_features_to_select=1로 설정하여 전체 21개 feature 전부의 ranking을 구할 수 있도록 하겠습니다. 그 다음 sel.ranking_을 통해서 중요한 feature의 ranking을 알아보겠습니다. 최종적으로는, ranking의 순서대로 해당되는 feature의 이름을 출력하겠습니다.
* 추가적으로, 출력 과정에서 feature_ranked라는 list를 만들어 feature selection 순서대로 feature를 저장해 놓겠습니다. 뒤에서 최종 모델 선정을 편하게 하기 위한 과정입니다.


In [117]:
# RFE를 통해 feature selection
from sklearn.feature_selection import RFE

select = RFE(RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=1)
sel = select.fit(x_train, y_train.values.ravel())
# feature selection 결과 ranking 출력
sel.ranking_

array([19, 18, 15, 10,  1, 21,  9,  4,  2, 13,  5, 11,  8, 17, 14, 20, 16,
        6,  3,  7, 12])

In [127]:
# ranking의 순서대로 feature 출력하기
temp = list(sel.ranking_)
feature_ranked = []
n = 1
for i in range(21):
    index = temp.index(n)
    result = data.columns[index+1]
    print(result)
    n += 1
    feature_ranked.append(result)

odor
gill-color
spore-print-color
gill-size
stalk-root
ring-type
population
stalk-surface-below-ring
gill-spacing
bruises
stalk-surface-above-ring
habitat
stalk-shape
stalk-color-below-ring
cap-color
ring-number
stalk-color-above-ring
cap-surface
cap-shape
veil-color
gill-attachment


* 위의 결과는 feature selection을 거쳐 중요한 순서대로 feature를 출력한 것입니다. 즉, 맨 위의 odor가 가장 중요한 feature이고, 맨 아래의 gill-attachment가 가장 중요하지 않은 feature입니다.


* 인상적인 점은 OneR 모델 실행을 위해 선택했던 best-feature의 결과가 cap-color이었다는 점입니다. 보다 엄밀한 feature selection의 과정을 거친 결과 cap-color은 하위권에 위치하는 feature고, odor가 1위의 feature라고 알게 되었습니다.

* 이 결과를 바탕으로 이제 중요한 feature를 선별하여 다시 모델을 훈련해보겠습니다.

### <font color = green>3. (6): feature selection, parameters 조정을 종합하기
** 우선 feature selection 결과 1등인 odor feature를 바탕으로 다시 OneR을 실행해 보겠습니다.

** 두 번째로, feature selection에 앞서 parameters를 조정했습니다. 그 결과 선정된 모델과 parameters는 다음과 같습니다.
* Logistic Regression: max_iter = 1000, penalty = 'none'
* MLP: default
* Random Forest: default

** 선정된 parameters에 대해 상위 10개의 feature를 바탕으로 다시 학습하여 결과를 출력하겠습니다. feature selection 전후 점수가 상승하는지 하락하는지 관찰해 보겠습니다. 그렇게 한다면 feature selection, parameters 조정을 종합하여 최종 모델을 선별할 수 있을 것입니다.


먼저 best feature odor에 대해 OneR을 실행했습니다.

In [121]:
best_feature = 'odor'
oneRules = []
for val in data[best_feature].unique():
    print(f"{best_feature} : {val}","-> ",end='')
    print(Counter(data[data[best_feature]==val]['class']).most_common()[0][0])
    oneRules.append((best_feature, val, 
                    Counter(data[data[best_feature]==val]['class']).most_common()[0][0]))

odor : 6 -> 1
odor : 0 -> 0
odor : 3 -> 0
odor : 5 -> 0
odor : 2 -> 1
odor : 1 -> 1
odor : 8 -> 1
odor : 7 -> 1
odor : 4 -> 1


* 다음으로는 최종 모델을 선정하겠습니다. 앞서 저장해 놓은 feature_ranked list에서 상위 10개의 feature만 사용하여 모델을 다시 훈련하겠습니다.
* sel_data라는 DataFrame을 만들어 상위 10개 feature를 저장했고, 그것을 바탕으로 훈련하겠습니다.

In [139]:
# feature selection 결과 상위 10개 feature 저장
sel_data = pd.DataFrame()
sel_data['class'] = data['class']
for i in range(10):
    sel_data[feature_ranked[i]] = data[feature_ranked[i]]
sel_data.head()

,class,odor,gill-color,spore-print-color,gill-size,stalk-root,ring-type,population,stalk-surface-below-ring,gill-spacing,bruises
0,1,6,4,2,1,3,4,3,2,0,1
1,0,0,4,3,0,2,4,2,2,0,1
2,0,3,5,3,0,2,4,2,2,0,1
3,1,6,5,2,1,3,4,3,2,0,1
4,0,5,4,3,0,3,0,0,2,1,0


In [140]:
# feature selection, parameters 조정이 반영된 모델 생성
y = sel_data[['class']]
x = sel_data.drop(columns = ['class'])

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
LR = LogisticRegression(max_iter = 1000, penalty = 'none')
MLP = MLPClassifier()
RF = RandomForestClassifier()

# logistic regression model 생성
mdl_LR = LR.fit(x_train, y_train.values.ravel())

# MLP 모델 생성
mdl_MLP = MLP.fit(x_train, y_train.values.ravel())

# Random Forest 모델 생성
mdl_RF = RF.fit(x_train, y_train.values.ravel())

# 각 모델의 점수 계산
print("Logistic Regression score: ", mdl_LR.score(x_test, y_test))
print("MLP score: ", mdl_MLP.score(x_test, y_test))
print("Random Forest score: ", mdl_RF.score(x_test, y_test))

Logistic Regression score:  0.9458572600492207
MLP score:  1.0
Random Forest score:  1.0


* 의외로, feature selection 결과 상위 10개의 feature를 적용하자 Logistic Regression의 성능이 떨어졌습니다. MLP, Random Forest는 워낙 성능이 좋았기 때문에 떨어지지 않았습니다.
* 어쨌든 feature selection은 부정적인 영향을 미쳤다고 판단하겠습니다. 또한 그렇기 때문에 MLP와 Random Forest의 최종 모델 역시 feature selection을 적용하지 않는 것으로 하겠습니다.

### <font color = green> 3. (7): 최종 모델 선정
* 최종 모델은 feature selection을 적용하지 않은 것으로 하겠습니다. 또한 위에서 조정한 parameters를 그대로 사용하겠습니다.

In [141]:
# feature selection, parameters 조정이 반영된 모델 생성
y = data[['class']]
x = data.drop(columns = ['class'])

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
LR = LogisticRegression(max_iter = 1000, penalty = 'none')
MLP = MLPClassifier()
RF = RandomForestClassifier()

# logistic regression model 생성
mdl_LR = LR.fit(x_train, y_train.values.ravel())

# MLP 모델 생성
mdl_MLP = MLP.fit(x_train, y_train.values.ravel())

# Random Forest 모델 생성
mdl_RF = RF.fit(x_train, y_train.values.ravel())

# 각 모델의 점수 계산
print("Logistic Regression score: ", mdl_LR.score(x_test, y_test))
print("MLP score: ", mdl_MLP.score(x_test, y_test))
print("Random Forest score: ", mdl_RF.score(x_test, y_test))

Logistic Regression score:  0.9721082854799016
MLP score:  1.0
Random Forest score:  1.0


cross validation 점수는 다음과 같습니다.

In [142]:
from sklearn.model_selection import cross_val_score

score1 = cross_val_score(LR, x_train, y_train.values.ravel(), cv = 5)
score2 = cross_val_score(MLP, x_train, y_train.values.ravel(), cv = 5)
score3 = cross_val_score(RF, x_train, y_train.values.ravel(), cv = 5)
print("cv_LR score: ", score1)
print("cv_MLP score: ", score2)
print("cv_RF score: ", score3)

cv_LR score:  [0.96309315 0.95778364 0.97009675 0.96042216 0.96833773]
cv_MLP score:  [1.         1.         1.         0.99912049 1.        ]
cv_RF score:  [1. 1. 1. 1. 1.]


# <font color=red> 4. 실험 결과 분석
우선 최종 모델은 다음과 같습니다.
* OneR: odor feature로 실행
* Logistic Regression: max_iter = 1000, penalty = 'none' 
--------------------------------------------------------- default model score: 0.949   --> 최종 모델  score: 0.972
* MLP: default
--------------------------------------------------------- default model score: 1,0   --> 최종 모델  score: 1.0
* Random Forest: default --> score: 1.0
--------------------------------------------------------- default model score: 1.0   --> 최종 모델  score: 1.0

* feature selection은 적용하지 않고 모든 feature를 다 사용한다. (unique한 값이 하나 뿐인 veil-type만 제외)
    


    
* 위의 모델들의 점수는 전부 test set에 대한 평가이며, cross validation의 결과도 크게 다르지 않다는 점을 위에 보였습니다. 조정이 필요 없는 MLP, RF는 논외로 하고, Logistic Regression에 대해서는 상당한 성능 향상을 이뤘습니다.

<font color = blue>1. 그렇다면 우선 default 모델에 대해 분석하겠습니다.</font>

왜 Logistic Regression은 MLP와 RF에 비해 성능이 좋지 않게 나타났을까요? 답은 데이터에 non-linear한 모델이 좀더 적합하다는 것입니다. 물론 LR의 성능인 0.949도 정말 준수한 건 맞습니다. 그러나 완벽하게 세밀한 분류를 하기 위해서는 확실히 non-linear한 모델이 필요할 것입니다.
    
<font color = blue>2. ensemble 모델의 성능 분석</font>

ensemble 모델인 Random Forest는 1.0의 성능을 보였습니다. 그러나 ensemble이 아닌 MLP역시 1.0을 보였기 때문에, ensemble 모델의 성능이 더 월등함을 밝혀 내지는 못했습니다. 그나마, cross validation을 하면 데이터에 따라 MLP에서 0.99의 점수가 나타나기도 하였으나, 이는 유의미한 성능 차이라고 보기는 어려울 것입니다. mushroom data에서는 단순히 non-linear 모델을 적용하는 것만으로 완벽한 성능을 가져왔으나, 더 복잡한 데이터에서는 분명 ensemble 모델이 더 좋은 성능을 보일 것으로 기대합니다.

<font color = blue>3. parameters 조정 분석 (왜 penalty='none'이 무조건 성능 향상을 가져왔나?)</font>
    

사실상 parameters 조정의 소득은 3개 모델을 통틀어 LR의 penalty='none' 뿐 입니다. 아쉽긴 하지만 MLP와 RF에 관해서는 parameters 조정이 필요하지 않았습니다. 성능을 깎아먹는 parameters라도 찾아보려고 하였으나, 모델의 성능이 너무 좋아서 악조건에서도 그대로 1.0의 점수를 유지했습니다.
그래도 Logistic Regression의 penalty='none'은 거의 무조건 성능 향상을 가져 왔습니다.
* penalty='none'이 성능을 향상시킨 이유를 추론하자면 다음과 같습니다. 제가 다른 수업에서 배운 바에 의하면 L1-norm, L2-norm 등은 regularization 방법입니다. 즉 모델을 generalize하고, overfitting을 줄이는 방법이라고 배웠습니다. generalize한다는 것에 관점을 바꾸면, "noise를 넣는다, 혼란을 준다"라고 볼 수 있을 것 같습니다. 정말 overfitting이 발생한 상황이라면 이것은 분명 도움이 됩니다. 그리고 대부분 실생활의 데이터는 이번 실험의 데이터보다 훨씬 광범위하고 복잡하기 때문에 overfitting이 흔하게 발생할 것으로 예상합니다.
* 그러나 이번 실험에서는 overfitting이 거의 전혀 일어나지 않았다고 판단했습니다. 실제로 점수를 봤을 때 train set의 점수가 특별히 월등히 높거나 그렇지 않았습니다. 이처럼 overfitting이 거의 전혀 일어나지 않은, 성능이 매우 우수한 모델에 대해서는 regularization이 오히려 방해가 될 수 있다고 생각했습니다. 
* 쉽게 말해, 그냥 default 상태에서 잘 학습하는 Logistic Regression 모델에, 괜히 noise를 넣어서 오히려 학습을 방해한 셈이죠. penalty='none'을 했다는 것은 곧 그 noise를 제거한 것이라고 생각했습니다. 그렇기 때문에 확실히 성능 향상을 가져왔다고 추론했습니다.

<font color = blue>4. feature selection 분석</font>
 
feature selection을 적용하자 Logistic Regression의 성능이 0.948까지 낮아졌습니다. parameters의 조정을 마쳐 0.972의 성능까지 기록한 모델에 적용한 것이니, 하락이 상당히 크다고 볼 수 있습니다.
* 수업 시간에 feature selection은 거의 무조건 성능 향상을 불러올 것이라고 배웠고, 저도 그런 결과가 나올 것이라고 예상하였습니다. 그래서 왜 성능 향상이 나타나지 않았는지 나름대로 곰곰이 생각해봤습니다. 제가 내린 답은, "모델의 성능이 너무 좋기 때문"입니다.

* 모델의 성능이 0.95이상 1.0수준으로 말도 안되게 좋은 상황이라면, 그것의 원인은 전적으로 모델 자체에만 있다고 보긴 힘들 것입니다. 그만큼 feature도 성능에 크게 기여한 것이라고 봐야 할 것입니다. 즉 feature과 모델의 상호작용이 완벽한 점수를 이뤄냈다고 생각했습니다. 
* 이러한 상황에서 굳이 상대적으로 기여도가 낮은 feature를 제거하는 게 부정적인 영향을 미쳤습니다. 기여도가 낮지만 0.948을 0.972로 만드는 작은 기여는 하고 있었던 것이겠지요. 상대적으로는 irrelevant하나, 어쨌든 조금이라도 긍정적인 영향을 주고 있는 feature들을 제거한 것이었습니다.
    
<font color = blue> 5. 더 실험해보고 싶은 사항</font>
* 실험을 마치고 분석하며 생각해보니, feature selection의 개수를 10개로 한 것이 성능 하락의 원인이었나도 추측해보게 되었습니다. 막연히 21개의 feature의 절반인 10개를 선택한 것이었고, 그 막연한 선택조차도 긍정적인 영향을 줄 것으로 기대했습니다.
* 그러나 성능 하락이 관찰되었습니다. 이러한 경우 다음부터는 feature selection의 개수를 바꿔가며 분석해 보는 것이 좋겠다고 생각했습니다. 10개를 고정하지 말고, [5, 6, 7, .... 20] 이렇게 숫자를 바꿔가면서 최적의 feature 개수까지 찾아봤으면 어땠을까 싶습니다. 그렇게 했다면 Logistic Regression의 0.972를 조금이라도 더 올려볼 수 있지 않았을까 싶습니다.


# <font color=red> 5. 결론 </font>
* mushroom data에 대해 4가지 모델(OneR, Logistic Regression, MLP, Random Forest)을 적용하였습니다.
* parameters 조정 결과 Logistic Regression에서 penalty='none'으로 조정하는 것이 거의 무조건 성능 향상을 보였습니다.
* MLP와 Random Forest는 default 모델에서 1.0의 성능이었습니다. parameters 조정을 시도해 봤으나, 성공적이지 않았습니다.
* feature selection은 성능 하락의 원인이 되었습니다.